# Transformer Encoder Đa Nhiệm: Phân loại Hình ảnh & Văn bản

Notebook này triển khai kiến trúc Transformer Encoder thống nhất có khả năng thực hiện cả Phân loại Hình ảnh (CIFAR-10) và Phân loại Văn bản (Bộ dữ liệu tùy chỉnh).

In [1]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, Dataset
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import Vocab
from collections import Counter
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from tqdm import tqdm
import numpy as np
import random
import os
import glob
import matplotlib.pyplot as plt
import torchvision.transforms.functional as TF

# Kiểm tra thiết bị
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Đang sử dụng thiết bị: {device}")

# Thiết lập Seed
def set_seed(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)

set_seed()

Đang sử dụng thiết bị: cuda


1.  **PatchEmbedding**: Để xử lý hình ảnh (theo phong cách Vision Transformer).
2.  **TextEmbedding**: Để xử lý văn bản (Token embeddings + Positional embeddings).
3.  **TransformerEncoderBackbone**: Bộ mã hóa Transformer cốt lõi được chia sẻ.
4.  **Task Heads**: Các đầu ra cụ thể cho từng tác vụ phân loại.

In [2]:
class PatchEmbedding(nn.Module):
    """Nhúng hình ảnh thành Patch (cho Vision Transformer)"""
    def __init__(self, in_channels=3, patch_size=4, emb_size=256, img_size=64):
        super().__init__()
        self.patch_size = patch_size
        self.proj = nn.Conv2d(in_channels, emb_size, kernel_size=patch_size, stride=patch_size)
        self.cls_token = nn.Parameter(torch.randn(1, 1, emb_size))
        self.pos_embedding = nn.Parameter(torch.randn(1, (img_size // patch_size)**2 + 1, emb_size))

    def forward(self, x):
        B = x.size(0)
        x = self.proj(x)  # [B, emb_size, H/patch, W/patch]
        x = x.flatten(2).transpose(1, 2)  # [B, num_patches, emb_size]
        cls_tokens = self.cls_token.expand(B, -1, -1)
        x = torch.cat([cls_tokens, x], dim=1)
        x = x + self.pos_embedding
        return x

class TextEmbedding(nn.Module):
    """Nhúng Token văn bản + Nhúng vị trí"""
    def __init__(self, vocab_size, embed_dim, max_len=256, padding_idx=1):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, embed_dim, padding_idx=padding_idx)
        self.pos_embedding = nn.Embedding(max_len, embed_dim)
        self.max_len = max_len

    def forward(self, x):
        # x: [batch_size, seq_len]
        positions = torch.arange(0, x.size(1), device=x.device).unsqueeze(0)
        x = self.embedding(x) + self.pos_embedding(positions)
        return x

class MultiTaskTransformer(nn.Module):
    def __init__(self, task_type='image', **kwargs):
        super().__init__()
        self.task_type = task_type
        self.emb_size = kwargs.get('emb_size', 256)
        self.num_classes = kwargs.get('num_classes', 10)
        self.dropout = kwargs.get('dropout', 0.1)
        
        # 1. Lớp Embedding
        if task_type == 'image':
            self.embedding = PatchEmbedding(
                in_channels=kwargs.get('in_channels', 3),
                patch_size=kwargs.get('patch_size', 4),
                emb_size=self.emb_size,
                img_size=kwargs.get('img_size', 64)
            )
        elif task_type == 'text':
            self.embedding = TextEmbedding(
                vocab_size=kwargs.get('vocab_size', 10000),
                embed_dim=self.emb_size,
                max_len=kwargs.get('max_len', 256),
                padding_idx=kwargs.get('padding_idx', 1)
            )
            self.attention_pool = nn.Linear(self.emb_size, 1) # Specific for text pooling
        
        # 2. Bộ mã hóa Transformer được chia sẻ
        encoder_layer = nn.TransformerEncoderLayer(
            d_model=self.emb_size, 
            nhead=kwargs.get('num_heads', 8),
            dim_feedforward=kwargs.get('hidden_dim', 512), 
            dropout=self.dropout, 
            batch_first=True,
            norm_first=True
        )
        self.transformer_encoder = nn.TransformerEncoder(encoder_layer, num_layers=kwargs.get('num_layers', 6))
        
        # 3. Đầu ra phân loại
        self.fc = nn.Linear(self.emb_size, self.num_classes)

    def forward(self, x):
        # x có thể là tensor ảnh hoặc chỉ số văn bản
        if self.task_type == 'text':
            padding_mask = (x == 1) # Giả sử PAD_IDX = 1
            x = self.embedding(x)
            x = self.transformer_encoder(x, src_key_padding_mask=padding_mask)
            
            # Attention Pooling cho văn bản
            attn_weights = torch.softmax(self.attention_pool(x), dim=1)
            x = torch.sum(x * attn_weights, dim=1)
            
        else: # Image
            x = self.embedding(x)
            x = self.transformer_encoder(x)
            x = x[:, 0, :] # Token CLS cho hình ảnh
            
        return self.fc(x)

In [3]:
# --- Tải Dữ liệu Hình ảnh ---
def get_image_loaders(batch_size=64):
    transform_train = transforms.Compose([
        transforms.Resize((64, 64)),
        transforms.RandomHorizontalFlip(),
        transforms.RandomCrop(64, padding=4),
        transforms.ToTensor(),
        transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
    ])

    transform_test = transforms.Compose([
        transforms.Resize((64, 64)),
        transforms.ToTensor(),
        transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
    ])

    train_dataset = torchvision.datasets.CIFAR10(root='./data', train=True, transform=transform_train, download=True)
    test_dataset = torchvision.datasets.CIFAR10(root='./data', train=False, transform=transform_test, download=True)

    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    test_loader = DataLoader(test_dataset, batch_size=batch_size)
    
    return train_loader, test_loader, train_dataset.classes

# --- Tải Dữ liệu Văn bản ---
class TextDataset(Dataset):
    def __init__(self, data, tokenizer, vocab, max_len=256):
        self.data = data
        self.tokenizer = tokenizer
        self.vocab = vocab
        self.max_len = max_len
        self.unk_idx = vocab['<unk>']
        self.pad_idx = vocab['<pad>']
    
    def encode(self, text):
        tokens = self.tokenizer(text)
        ids = [self.vocab[token] if token in self.vocab.stoi else self.unk_idx for token in tokens][:self.max_len]
        if len(ids) < self.max_len:
            ids += [self.pad_idx] * (self.max_len - len(ids))
        return ids

    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        text, label = self.data[idx]
        return torch.tensor(self.encode(text), dtype=torch.long), torch.tensor(label, dtype=torch.long)

def get_text_loaders(data_dir='TextData', batch_size=64):
    # 1. Tải dữ liệu thô
    classes = sorted([d for d in os.listdir(data_dir) if os.path.isdir(os.path.join(data_dir, d))])
    label_map = {class_name: idx for idx, class_name in enumerate(classes)}
    
    all_data = []
    for class_name in classes:
        class_dir = os.path.join(data_dir, class_name)
        file_paths = glob.glob(os.path.join(class_dir, '*.txt'))
        for file_path in file_paths:
            try:
                with open(file_path, 'r', encoding='utf-16') as f:
                    text = f.read().strip()
                    if text:
                        all_data.append((text, label_map[class_name]))
            except Exception as e:
                pass # Bỏ qua lỗi
                
    train_data, test_data = train_test_split(all_data, test_size=0.2, random_state=42, stratify=[label for _, label in all_data])
    
    # 2. Xây dựng bộ từ vựng
    tokenizer = get_tokenizer("basic_english")
    counter = Counter()
    for text, _ in train_data:
        counter.update(tokenizer(text))
    vocab = Vocab(counter, specials=['<unk>', '<pad>'])
    
    # 3. Tạo Dataset
    train_dataset = TextDataset(train_data, tokenizer, vocab)
    test_dataset = TextDataset(test_data, tokenizer, vocab)
    
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    test_loader = DataLoader(test_dataset, batch_size=batch_size)
    
    return train_loader, test_loader, classes, vocab

In [4]:
def train(model, loader, criterion, optimizer):
    model.train()
    total_loss = 0
    loop = tqdm(loader, desc="Training", leave=False)
    for x, y in loop:
        x, y = x.to(device), y.to(device)
        optimizer.zero_grad()
        out = model(x)
        loss = criterion(out, y)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
        optimizer.step()
        total_loss += loss.item()
        loop.set_postfix(loss=loss.item())
    return total_loss / len(loader)

def evaluate(model, loader, criterion):
    model.eval()
    preds, targets = [], []
    total_loss = 0
    loop = tqdm(loader, desc="Evaluating", leave=False)
    with torch.no_grad():
        for x, y in loop:
            x, y = x.to(device), y.to(device)
            out = model(x)
            loss = criterion(out, y)
            total_loss += loss.item()
            pred = out.argmax(dim=1)
            preds.extend(pred.cpu().numpy())
            targets.extend(y.cpu().numpy())
    return total_loss / len(loader), accuracy_score(targets, preds)

Chọn tác vụ chạy bằng cách thiết lập `TASK_TYPE`.
- `TASK_TYPE = 'image'` cho Phân loại CIFAR-10.
- `TASK_TYPE = 'text'` cho Phân loại Văn bản.

In [5]:
# CẤU HÌNH
TASK_TYPE = 'text' # Tùy chọn: 'image', 'text'
NUM_EPOCHS = 10

print(f"Bắt đầu tác vụ: {TASK_TYPE.upper()}")

if TASK_TYPE == 'image':
    # 1. Tải Dữ liệu
    train_loader, test_loader, classes = get_image_loaders()
    num_classes = len(classes)
    
    # 2. Khởi tạo Mô hình
    model = MultiTaskTransformer(
        task_type='image',
        emb_size=256,
        num_heads=8,
        hidden_dim=512,
        num_layers=6,
        num_classes=num_classes,
        dropout=0.1
    ).to(device)
    
    # 3. Bộ tối ưu hóa
    optimizer = torch.optim.AdamW(model.parameters(), lr=3e-4, weight_decay=1e-4)
    scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=NUM_EPOCHS)

elif TASK_TYPE == 'text':
    # 1. Tải Dữ liệu
    train_loader, test_loader, classes, vocab = get_text_loaders()
    num_classes = len(classes)
    vocab_size = len(vocab)
    pad_idx = vocab['<pad>']
    
    # 2. Khởi tạo Mô hình
    model = MultiTaskTransformer(
        task_type='text',
        vocab_size=vocab_size,
        emb_size=256,
        num_heads=4,
        hidden_dim=512,
        num_layers=2,
        num_classes=num_classes,
        dropout=0.3,
        padding_idx=pad_idx
    ).to(device)
    
    # 3. Bộ tối ưu hóa
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-4)
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=2)

criterion = nn.CrossEntropyLoss()

# 4. Vòng lặp Huấn luyện
best_acc = 0
for epoch in range(NUM_EPOCHS):
    print(f"\nEpoch {epoch+1}/{NUM_EPOCHS}")
    train_loss = train(model, train_loader, criterion, optimizer)
    test_loss, test_acc = evaluate(model, test_loader, criterion)
    
    if TASK_TYPE == 'image':
        scheduler.step()
    else:
        scheduler.step(test_loss)
        
    print(f"Loss: {train_loss:.4f} - Test Loss: {test_loss:.4f} - Test Accuracy: {test_acc:.4f}")
    
    if test_acc > best_acc:
        best_acc = test_acc
        torch.save(model.state_dict(), f"multi_task_transformer_{TASK_TYPE}.pth")
        print("Đã lưu mô hình tốt nhất!")

Bắt đầu tác vụ: TEXT


c:\Users\nguye\AppData\Local\Programs\Python\Python312\Lib\site-packages\torch\nn\modules\transformer.py:379: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.norm_first was True
  warnings.warn(



Epoch 1/10


Loss: 1.5688 - Test Loss: 0.9021 - Test Accuracy: 0.7151
Đã lưu mô hình tốt nhất!

Epoch 2/10


Loss: 0.6941 - Test Loss: 0.6316 - Test Accuracy: 0.7976
Đã lưu mô hình tốt nhất!

Epoch 3/10


...

Loss: 0.2162 - Test Loss: 0.5858 - Test Accuracy: 0.8247

Epoch 7/10


Loss: 0.1757 - Test Loss: 0.6775 - Test Accuracy: 0.8237

Epoch 8/10


Loss: 0.1495 - Test Loss: 0.6774 - Test Accuracy: 0.8247

Epoch 9/10


Loss: 0.0702 - Test Loss: 0.6641 - Test Accuracy: 0.8539
Đã lưu mô hình tốt nhất!

Epoch 10/10


Loss: 0.0378 - Test Loss: 0.7389 - Test Accuracy: 0.8445


## 6. Dự đoán (Inference)

Chạy dự đoán trên một mẫu dựa trên tác vụ đã chọn.

In [8]:
model.eval()
if TASK_TYPE == 'image':
    # Dự đoán Hình ảnh
    def unnormalize(tensor):
        mean = torch.tensor((0.4914, 0.4822, 0.4465)).view(3, 1, 1)
        std = torch.tensor((0.2023, 0.1994, 0.2010)).view(3, 1, 1)
        tensor = tensor.cpu() * std + mean
        return tensor.clamp(0, 1)

    idx = random.randint(0, len(test_loader.dataset) - 1)
    img, label = test_loader.dataset[idx]
    
    with torch.no_grad():
        out = model(img.unsqueeze(0).to(device))
        pred = out.argmax(dim=1).item()
        conf = out.softmax(dim=1).max().item()
        
    plt.imshow(TF.to_pil_image(unnormalize(img)))
    plt.title(f"Dự đoán: {classes[pred]} ({conf:.2f}) | Thực tế: {classes[label]}")
    plt.axis('off')
    plt.show()

elif TASK_TYPE == 'text':
    # Dự đoán Văn bản
    sample_texts = [
        "Đội tuyển bóng đá Việt Nam đã giành chiến thắng thuyết phục trước đối thủ.",
        "Thị trường chứng khoán hôm nay ghi nhận mức tăng điểm kỷ lục.",
        "Ca sĩ Mỹ Tâm vừa ra mắt album mới với nhiều ca khúc hit.",
        "Giá vàng trong nước tiếp tục tăng cao do ảnh hưởng của thị trường thế giới.",
        "Các nhà khoa học vừa phát hiện một loài động vật mới tại vườn quốc gia.",
        "Bộ Giáo dục và Đào tạo công bố phương án thi tốt nghiệp THPT năm nay.",
        "Du lịch Việt Nam đang thu hút ngày càng nhiều khách quốc tế.",
        "Món phở Hà Nội luôn là niềm tự hào của ẩm thực Việt Nam.",
        "Tình hình giao thông tại các thành phố lớn đang ngày càng ùn tắc."
    ]
    
    print("--- Dự đoán Văn bản ---")
    with torch.no_grad():
        for text in sample_texts:
            # Tokenize & Mã hóa
            tokenizer = get_tokenizer("basic_english")
            tokens = tokenizer(text)
            ids = [vocab[token] if token in vocab.stoi else vocab['<unk>'] for token in tokens][:256]
            if len(ids) < 256:
                ids += [vocab['<pad>']] * (256 - len(ids))
            
            tensor = torch.tensor(ids, dtype=torch.long).unsqueeze(0).to(device)
            out = model(tensor)
            pred = out.argmax(dim=1).item()
            print(f"Văn bản: {text}\nDự đoán: {classes[pred]}\n")

--- Dự đoán Văn bản ---
Văn bản: Đội tuyển bóng đá Việt Nam đã giành chiến thắng thuyết phục trước đối thủ.
Dự đoán: Bong da

Văn bản: Thị trường chứng khoán hôm nay ghi nhận mức tăng điểm kỷ lục.
Dự đoán: Kinh doanh quoc te

Văn bản: Ca sĩ Mỹ Tâm vừa ra mắt album mới với nhiều ca khúc hit.
Dự đoán: Am nhac

Văn bản: Giá vàng trong nước tiếp tục tăng cao do ảnh hưởng của thị trường thế giới.
Dự đoán: Kinh doanh quoc te

Văn bản: Các nhà khoa học vừa phát hiện một loài động vật mới tại vườn quốc gia.
Dự đoán: Kinh doanh quoc te

Văn bản: Bộ Giáo dục và Đào tạo công bố phương án thi tốt nghiệp THPT năm nay.
Dự đoán: Giao duc

Văn bản: Du lịch Việt Nam đang thu hút ngày càng nhiều khách quốc tế.
Dự đoán: Thoi trang

Văn bản: Món phở Hà Nội luôn là niềm tự hào của ẩm thực Việt Nam.
Dự đoán: Thoi trang

Văn bản: Tình hình giao thông tại các thành phố lớn đang ngày càng ùn tắc.
Dự đoán: Giai tri tin hoc

